In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import numpy as np
import pandas as pd
import keras
from keras.models import *
from keras.layers import *
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras import backend as K
K.set_session(session)

/hdd/home/peter0749/virtualenv/tf_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/hdd/home/peter0749/virtualenv/tf_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
DATASET_ROOT = '/hdd/dataset/tbrain/'
TRAIN_PATH = os.path.join(DATASET_ROOT, 'taetfp_utf8.csv')
TS_IN=260
TS_OUT=5

In [3]:
train_csv = pd.read_csv(TRAIN_PATH)
train_csv.sort_values(by=['日期'], ascending=True, inplace=True)
train_csv.head(n=10)

,代碼,日期,中文簡稱,開盤價(元),最高價(元),最低價(元),收盤價(元),成交張數(張)
0,50,20130102,元大台灣50,46.57,47.13,46.49,46.92,"16,487"
11574,59,20130102,富邦金融,24.65,24.65,24.65,24.65,2
14146,6203,20130102,元大MSCI台灣,22.61,22.61,22.56,22.56,28
16718,6208,20130102,富邦台50,27.00,27.33,26.99,27.21,79
7716,56,20130102,元大高股息,18.45,18.63,18.45,18.55,"1,623"
2572,52,20130102,富邦科技,27.45,27.85,27.45,27.81,26
15432,6204,20130102,永豐臺灣加權,33.61,33.95,33.53,33.86,"1,574"
9002,57,20130102,富邦摩台,30.35,30.69,30.35,30.69,69
1286,51,20130102,元大中型100,22.36,22.66,22.36,22.65,277
10288,58,20130102,富邦發達,32.45,32.45,32.12,32.12,2


In [4]:
codes = list(sorted(set(train_csv['代碼'])))
fields = ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交張數(張)']

In [5]:
split_train = []
for code in codes:
    row = train_csv.loc[train_csv['代碼']==code][fields]
    row = row.applymap(lambda x: float(x.replace(',','')) if type(x)==str else x)
    split_train.append(np.array(row))

In [6]:
len(codes), len(split_train)

(18, 18)

In [7]:
split_train[0]

array([[   46.57,    47.13,    46.49,    46.92, 16487.  ],
       [   47.35,    47.48,    47.13,    47.31, 29020.  ],
       [   47.31,    47.31,    46.92,    47.  ,  9837.  ],
       ...,
       [   82.25,    82.35,    81.8 ,    82.1 ,  4099.  ],
       [   82.65,    83.05,    82.65,    82.85,  4994.  ],
       [   82.85,    83.05,    82.75,    82.95,   878.  ]])

In [8]:
from keras.regularizers import *

# custom R2-score metrics for keras backend
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def make_model(ts_in=60, ts_out=5, n_field=5, n_encode=128, n_decode=128, reg_a=0.1):
    input_ = Input(shape=(ts_in, n_field))
    lstm_1 = CuDNNLSTM(n_encode, return_sequences=False, recurrent_regularizer=l2(reg_a)) (input_)
    rep_vec_2 = RepeatVector(ts_out) (lstm_1)
    lstm_3 = CuDNNLSTM(n_decode, return_sequences=True, recurrent_regularizer=l2(reg_a)) (rep_vec_2)
    lstm_4 = CuDNNLSTM(n_decode, return_sequences=True, recurrent_regularizer=l2(reg_a)) (lstm_3)
    fc_4 = TimeDistributed(Dense(1, kernel_regularizer=l2(reg_a))) (lstm_4)
    return Model([input_], [fc_4])
make_model(ts_in=TS_IN, ts_out=TS_OUT).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 260, 5)            0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               69120     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 5, 128)            132096    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 5, 128)            132096    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 1)              129       
Total params: 333,441
Trainable params: 333,441
Non-trainable params: 0
_________________________________________________________________


In [9]:
def make_seq(seq, ts_in=60, ts_out=5):
    Xs = []
    Ys = []
    total_l = ts_in + ts_out
    for i in range(0, len(seq)-total_l+1):
        Xs.append(seq[i:i+ts_in])
        Ys.append(seq[i+ts_in:i+total_l, 3:4])
    return np.array(Xs), np.array(Ys)

In [10]:
train_18 = []
for seq in split_train:
    Xs, Ys = make_seq(seq, ts_in=TS_IN, ts_out=TS_OUT)
    train_18.append([Xs, Ys])

In [11]:
from keras.callbacks import *
from sklearn.model_selection import train_test_split
BATCH_SIZE = 64
EPOCH = 500
for model_n, (X, Y) in enumerate(train_18):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    prefix = str(model_n)+'_'
    model = make_model(ts_in=TS_IN, ts_out=TS_OUT)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[r2_keras])
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test) , batch_size=BATCH_SIZE, epochs=EPOCH, callbacks=[TensorBoard(log_dir=prefix+'logs')], shuffle=True)
    model.save(prefix+'model.h5')
    K.clear_session()

Train on 817 samples, validate on 205 samples
Epoch 1/500
817/817 [==============================] - 1s 1ms/step - loss: 4138.3313 - r2_keras: -48.3041 - val_loss: 3498.0622 - val_r2_keras: -40.1862
Epoch 2/500
817/817 [==============================] - 0s 478us/step - loss: 3205.0482 - r2_keras: -37.1549 - val_loss: 2706.0934 - val_r2_keras: -30.8011
Epoch 3/500
817/817 [==============================] - 0s 481us/step - loss: 2653.1130 - r2_keras: -29.9335 - val_loss: 2370.3905 - val_r2_keras: -26.8307
Epoch 4/500
817/817 [==============================] - 0s 476us/step - loss: 2372.3427 - r2_keras: -27.6033 - val_loss: 2145.1942 - val_r2_keras: -24.1869
Epoch 5/500
817/817 [==============================] - 0s 482us/step - loss: 2163.3538 - r2_keras: -24.4713 - val_loss: 1960.4528 - val_r2_keras: -22.0258
Epoch 6/500
817/817 [==============================] - 0s 478us/step - loss: 1985.9402 - r2_keras: -22.5099 - val_loss: 1800.6777 - val_r2_keras: -20.1576
Epoch 7/500
817/817 [=====

817/817 [==============================] - 0s 478us/step - loss: 136.5066 - r2_keras: -0.5755 - val_loss: 129.6173 - val_r2_keras: -0.4848
Epoch 55/500
817/817 [==============================] - 0s 479us/step - loss: 135.3715 - r2_keras: -0.5551 - val_loss: 129.1560 - val_r2_keras: -0.4793
Epoch 56/500
817/817 [==============================] - 0s 477us/step - loss: 134.5363 - r2_keras: -0.5275 - val_loss: 128.7485 - val_r2_keras: -0.4744
Epoch 57/500
817/817 [==============================] - 0s 475us/step - loss: 133.7580 - r2_keras: -0.5308 - val_loss: 128.4347 - val_r2_keras: -0.4705
Epoch 58/500
817/817 [==============================] - 0s 478us/step - loss: 133.1006 - r2_keras: -0.5271 - val_loss: 128.1985 - val_r2_keras: -0.4677
Epoch 59/500
817/817 [==============================] - 0s 478us/step - loss: 132.5241 - r2_keras: -0.5301 - val_loss: 128.0263 - val_r2_keras: -0.4656
Epoch 60/500
817/817 [==============================] - 0s 478us/step - loss: 132.0623 - r2_keras: -0

Epoch 108/500
817/817 [==============================] - 0s 481us/step - loss: 128.5862 - r2_keras: -0.4808 - val_loss: 127.8644 - val_r2_keras: -0.4643
Epoch 109/500
817/817 [==============================] - 0s 480us/step - loss: 128.5679 - r2_keras: -0.4813 - val_loss: 127.8506 - val_r2_keras: -0.4642
Epoch 110/500
817/817 [==============================] - 0s 480us/step - loss: 128.4301 - r2_keras: -0.4837 - val_loss: 126.7853 - val_r2_keras: -0.4517
Epoch 111/500
817/817 [==============================] - 0s 480us/step - loss: 127.6801 - r2_keras: -0.4841 - val_loss: 126.4901 - val_r2_keras: -0.4478
Epoch 112/500
817/817 [==============================] - 0s 483us/step - loss: 127.4089 - r2_keras: -0.4553 - val_loss: 125.8659 - val_r2_keras: -0.4403
Epoch 113/500
817/817 [==============================] - 0s 480us/step - loss: 127.1452 - r2_keras: -0.4668 - val_loss: 125.9173 - val_r2_keras: -0.4407
Epoch 114/500
817/817 [==============================] - 0s 481us/step - loss: 127

817/817 [==============================] - 0s 480us/step - loss: 109.6714 - r2_keras: -0.2482 - val_loss: 107.9429 - val_r2_keras: -0.2220
Epoch 162/500
817/817 [==============================] - 0s 476us/step - loss: 109.2965 - r2_keras: -0.2338 - val_loss: 107.5462 - val_r2_keras: -0.2171
Epoch 163/500
817/817 [==============================] - 0s 473us/step - loss: 108.7873 - r2_keras: -0.2317 - val_loss: 107.1162 - val_r2_keras: -0.2118
Epoch 164/500
817/817 [==============================] - 0s 478us/step - loss: 108.4122 - r2_keras: -0.2237 - val_loss: 106.6960 - val_r2_keras: -0.2068
Epoch 165/500
817/817 [==============================] - 0s 479us/step - loss: 108.0307 - r2_keras: -0.2228 - val_loss: 106.3177 - val_r2_keras: -0.2023
Epoch 166/500
817/817 [==============================] - 0s 483us/step - loss: 107.6527 - r2_keras: -0.2404 - val_loss: 106.3086 - val_r2_keras: -0.2020
Epoch 167/500
817/817 [==============================] - 0s 480us/step - loss: 107.2649 - r2_ker

Epoch 215/500
817/817 [==============================] - 0s 482us/step - loss: 94.6829 - r2_keras: -0.0527 - val_loss: 94.3789 - val_r2_keras: -0.0513
Epoch 216/500
817/817 [==============================] - 0s 481us/step - loss: 94.6136 - r2_keras: -0.0624 - val_loss: 94.5145 - val_r2_keras: -0.0528
Epoch 217/500
817/817 [==============================] - 0s 479us/step - loss: 94.5389 - r2_keras: -0.0583 - val_loss: 94.2656 - val_r2_keras: -0.0498
Epoch 218/500
817/817 [==============================] - 0s 479us/step - loss: 94.4135 - r2_keras: -0.0669 - val_loss: 94.4354 - val_r2_keras: -0.0517
Epoch 219/500
817/817 [==============================] - 0s 475us/step - loss: 94.3330 - r2_keras: -0.0601 - val_loss: 94.2669 - val_r2_keras: -0.0496
Epoch 220/500
817/817 [==============================] - 0s 478us/step - loss: 94.2232 - r2_keras: -0.0495 - val_loss: 94.3166 - val_r2_keras: -0.0499
Epoch 221/500
817/817 [==============================] - 0s 484us/step - loss: 94.1847 - r2_ke

...

817/817 [==============================] - 0s 481us/step - loss: 91.6416 - r2_keras: -0.0181 - val_loss: 92.3115 - val_r2_keras: -0.0204
Epoch 486/500
817/817 [==============================] - 0s 479us/step - loss: 91.6223 - r2_keras: -0.0105 - val_loss: 92.4467 - val_r2_keras: -0.0218
Epoch 487/500
817/817 [==============================] - 0s 481us/step - loss: 91.7179 - r2_keras: -0.0234 - val_loss: 92.7712 - val_r2_keras: -0.0253
Epoch 488/500
817/817 [==============================] - 0s 482us/step - loss: 91.7749 - r2_keras: -0.0294 - val_loss: 92.2355 - val_r2_keras: -0.0197
Epoch 489/500
817/817 [==============================] - 0s 483us/step - loss: 91.6381 - r2_keras: -0.0108 - val_loss: 92.3630 - val_r2_keras: -0.0210
Epoch 490/500
817/817 [==============================] - 0s 478us/step - loss: 91.6651 - r2_keras: -0.0337 - val_loss: 92.4204 - val_r2_keras: -0.0216
Epoch 491/500
817/817 [==============================] - 0s 481us/step - loss: 91.6233 - r2_keras: -0.0089 -

817/817 [==============================] - 0s 471us/step - loss: 11.8408 - r2_keras: -0.5809 - val_loss: 11.6250 - val_r2_keras: -0.5123
Epoch 40/500
817/817 [==============================] - 0s 471us/step - loss: 11.6599 - r2_keras: -0.5634 - val_loss: 11.4587 - val_r2_keras: -0.4929
Epoch 41/500
817/817 [==============================] - 0s 470us/step - loss: 11.4577 - r2_keras: -0.5489 - val_loss: 11.2749 - val_r2_keras: -0.4675
Epoch 42/500
817/817 [==============================] - 0s 469us/step - loss: 11.2812 - r2_keras: -0.5136 - val_loss: 11.1442 - val_r2_keras: -0.4501
Epoch 43/500
817/817 [==============================] - 0s 467us/step - loss: 11.1350 - r2_keras: -0.4955 - val_loss: 10.9319 - val_r2_keras: -0.4216
Epoch 44/500
817/817 [==============================] - 0s 468us/step - loss: 10.9316 - r2_keras: -0.4552 - val_loss: 10.7778 - val_r2_keras: -0.3981
Epoch 45/500
817/817 [==============================] - 0s 466us/step - loss: 10.7772 - r2_keras: -0.4335 - val_l

817/817 [==============================] - 0s 473us/step - loss: 8.0816 - r2_keras: -0.0337 - val_loss: 8.1620 - val_r2_keras: -0.0197
Epoch 95/500
817/817 [==============================] - 0s 472us/step - loss: 8.0723 - r2_keras: -0.0312 - val_loss: 8.1391 - val_r2_keras: -0.0161
Epoch 96/500
817/817 [==============================] - 0s 473us/step - loss: 8.0683 - r2_keras: -0.0365 - val_loss: 8.1411 - val_r2_keras: -0.0164
Epoch 97/500
817/817 [==============================] - 0s 475us/step - loss: 8.0663 - r2_keras: -0.0329 - val_loss: 8.1452 - val_r2_keras: -0.0171
Epoch 98/500
817/817 [==============================] - 0s 473us/step - loss: 8.0633 - r2_keras: -0.0295 - val_loss: 8.1499 - val_r2_keras: -0.0177
Epoch 99/500
817/817 [==============================] - 0s 474us/step - loss: 8.0627 - r2_keras: -0.0354 - val_loss: 8.1390 - val_r2_keras: -0.0162
Epoch 100/500
817/817 [==============================] - 0s 473us/step - loss: 8.0617 - r2_keras: -0.0245 - val_loss: 8.1406 

817/817 [==============================] - 0s 475us/step - loss: 7.9944 - r2_keras: -0.0098 - val_loss: 8.0879 - val_r2_keras: -0.0063
Epoch 150/500
817/817 [==============================] - 0s 471us/step - loss: 7.9950 - r2_keras: -0.0186 - val_loss: 8.0822 - val_r2_keras: -0.0056
Epoch 151/500
817/817 [==============================] - 0s 474us/step - loss: 7.9973 - r2_keras: -0.0158 - val_loss: 8.0767 - val_r2_keras: -0.0050
Epoch 152/500
817/817 [==============================] - 0s 474us/step - loss: 7.9970 - r2_keras: -0.0207 - val_loss: 8.0944 - val_r2_keras: -0.0071
Epoch 153/500
817/817 [==============================] - 0s 472us/step - loss: 7.9987 - r2_keras: -0.0195 - val_loss: 8.0845 - val_r2_keras: -0.0059
Epoch 154/500
817/817 [==============================] - 0s 474us/step - loss: 7.9939 - r2_keras: -0.0273 - val_loss: 8.0923 - val_r2_keras: -0.0069
Epoch 155/500
817/817 [==============================] - 0s 473us/step - loss: 7.9941 - r2_keras: -0.0073 - val_loss: 8.

KeyboardInterrupt: 